# Outline of thought process
- starting out with the creature class as the base class. The creature class will be used to create monsters/creatures that<br>
have not become npcs

- the pc_class class will be used to give player characters access to class mechanics
- the pc class will inherit from the creature class and the pc_class class to create a pc
- the npc class will inherit from the creature class and set the is_npc flag of an object to True so that a monster can be <br>
changed to npc status and stored to come back as a recurring character
- the item class will be used to create all items(including interactable objects our pcs might run into or use to change the <br> course of a battle. Think setting a fire to obscure the battle field or wedging a door shut


In [1]:
import pc_classes
import creatures
import races
import copy
import numpy as np
import random

from dice import Dice, DiceBag as DB, dice_avg
from skills import skills
from weapons import weapons as wpn
from armor import armor as ar

In [2]:
db = DB()
d = Dice()

In [3]:
class PINGenerator:
    def __init__(self):
        pass
    
    def generate_pin(self, obj_type):
        ## c = creatrues start = 0xa
        ## idf = intelegint design features start = 0xb
        ## nf = natural features = 0xc
        ## oo = other objects = 0xd
        pin_base = {'c': '0xa', 'idf': '0xb', 'nf': '0xc', 'oo': '0xd'}
        pin_elements = '0123456789abcdef'
        base = pin_base[obj_type]
        pin = base
        for _ in range(7):
            pin = pin + random.choice(pin_elements)
        return pin
        


In [4]:
### start of a creature class

class Creature():
    def __init__(self, creature, name = False):
        creature_attrs = creatures.creature_keys
        self.entity_type = 'monster'
        if name == False:
            self.name= creature['race']
        else:
            self.name= name
        for key in creature.keys():
            setattr(self, key, creature[key])
        for attr in creature_attrs:
            if hasattr(self, attr)==True:
                pass
            else:
                setattr(self, attr, False)
        self.disposition = 'hostile'
        self.life_state = 'alive'
        if hasattr(self, 'proficiencies') == False:
            self.proficiencies = []
            self.proficiencies.append(('unarmed_strike', 'r_trait'))
        else:
            self.proficiencies.append(('unarmed_strike', 'r_trait'))
            
    def __ability_mod(self, abl):
        return creatures.ability_mod_tbl[self.abilities[abl]]
    
    def __dual_weilder_check(self):
        dual_weilder = False
        for prof in self.proficiencies:
            if 'dual_weilder' in prof:
                dual_weilder = True
        return dual_weilder
    
    def __armor_disadvantage(self):
        if self.disadvantage == False:
            self.disadvantage = []
            
        self.disadvantage.append('dex')
        self.disadvantage.append('str')
        self.disadvantage.append('attack')
        self.spell_casting = False
        for skill in self.skills:
            if self.skills[skill]['ability']== 'str' or self.skills[skill]['ability']== 'dex':
                self.disadvantage.append(skill)
                
    def __is_finesse_wpn(self, wep):
        return 'finesse' in wpn[wep]['properties']
    
    def __has_prof_bon(self):
        if hasattr(self, 'proficiency_bonus'):
            return self.proficiency_bonus
        else:
            return 0
        
    # get monster attack bonus. not used for pcs
    def __has_attack_bon(self):
        if hasattr(self,'attack_bonus'):
            return self.attack_bonus
        else:
            return False
                
    def equip_armor(self, armor):
        ac_base = armor['armor_class']['base']
        ac_dex_bon = armor['armor_class']['dex_mod']
        ac_dex_max = armor['armor_class']['dex_mod_max']
        ac_str_req = armor['str_req']
        ac_stealth = armor['stealth_penalty']
        
        if self.armor == False:
            self.armor = armor['name']
            if ac_str_req != False:
                self.ac = ac_base
                if self.abilities['str'] < ac_str_req:
                    self.movement -= 10
            if ac_dex_bon == True:
                if ac_dex_max != False:
                    self.ac = ac_base + ac_dex_max
                else:
                    self.ac = ac_base + self.__ability_mod('dex')
            else:
                self.ac = ac_base
            
            for prof in self.proficiencies:
                if armor['type'] in prof:
                    pass
            else:
                self.__armor_disadvantage()

        else:
            print(self.armor + ' is already equiped.')
    
    def equip_shield(self, shield):
        if self.off_hand == False:
            self.ac += 2
            self.off_hand == shield['name']
            for prof in self.proficiencies:
                if shield['type'] in prof:
                    pass
            else:
                self.__armor_disadvantage()
        else:
            print(self.off_hand + ' is already equiped.')
            
    
    def equip_weapon(self, weapon, main_hand = True, off_hand = False, 
                     versatile=False,):
        
        dual_weilder = self.__dual_weilder_check()

        if off_hand == False and versatile == False:
            if self.main_hand != False:
                print(self.main_hand + ' is already equiped')
            else:
                if 'two_handed' in weapon['properties']:
                    self.main_hand = weapon['name']
                    self.off_hand = weapon['name']
                else:   
                    self.main_hand = weapon['name']
        if versatile == True:
            if self.off_hand != False:
                print(self.off_hand + 
                      ' is equipped in off_hand. Please unequip to use this weapons versatile properties')
            elif self.main_hand != False and self.main_hand != weapon['name']:
                print(self.main_hand +
                     ' is equipped. Please unequip this weapon to equip a new weapon')
            else:
                self.main_hand = weapon['name']
                self.off_hand = weapon['name']
                
        if off_hand == True:
            if self.off_hand != False:
                print(self.off_hand + ' is already equiped')
            elif self.main_hand != False:
                if'light' not in wpn[self.main_hand]['properties'] and dual_weilder == False:
                    print('You do not meet the requirements for dual weilding')
            elif 'light' not in weapon['proficiencies'] and dual_weilder == False:
                 print('You do not meet the requirements for dual weilding')
            else:
                self.off_hand = weapon['name']
        
    def unequip_weapon(self, main_hand = True, off_hand = False):
        if main_hand == True:
            self.main_hand = False
        if off_hand == True:
            self.off_hand = False   
    
    def damage_roll(self, wep):
        #dmg = d.roll_from_string(wpn[wep]['damage'])
        #hp = trgt.hp
        #new_hp = hp - dmg
        #trgt.hp = new_hp
        #return dmg
        str_mod = self.__ability_mod('str')
        dex_mod = self.__ability_mod('dex')
        if self.main_hand != False and self.main_hand == self.off_hand:
            if 'versatile' in wpn[wep]['properties']:
                return -(d.roll_from_string(wpn[wep]['two_handed_damage']) + str_mod)
            else:
                return -(d.roll_from_string(wpn[wep]['damage']) + str_mod)
        elif 'finesse' in wpn[wep]['properties']:
            return -(d.roll_from_string(wpn[wep]['damage']) + max[str_mod, dex_mod])
        elif wpn[wep]['type'] == 'ranged':
            return -(d.roll_from_string(wpn[wep]['damage']) + dex_mod)
        else:
            return -(d.roll_from_string(wpn[wep]['damage']) + str_mod)


    #def un-equip_armor(self)
    #def add_inventory(self, itm):
    #def cast(self, spell):
        #if spell.r_attack == True:
            #attack(spell)
        #elif spell.r_save == True:
        #else:
            #do a thing
    def attack_roll(self, wep, a_type = False):
        str_mod = self.__ability_mod('str')
        dex_mod = self.__ability_mod('dex')
        prof_bon = self.__has_prof_bon()
        
        if self.__has_attack_bon():
            return db.d20.roll() + self.attack_bonus
        else:
            if self.__is_finesse_wpn(wep):
                return db.d20.roll() + prof_bon + max[str_mod, dex_mod]
            elif a_type == 'spell':
                c_abil_mod = creatures.ability_mod_tbl[self.abilities[self.cast_ability]]
                return db.d20.roll() + prof_bon + c_abil_mod
            elif a_type == 'ranged':
                return db.d20.roll() + prof_bon + dex_mod
            else:
                return db.d20.roll() + prof_bon + str_mod
            
    def saving_throw(self, save_stat):
        prof = False
        abl_mod = self.__ability_mod(save_stat)
        
        if hasattr(self, 'saving_throws') == False:
            return db.d20.roll() + abl_mod
        else:
            for prof in self.saving_throws:
                if save_stat in prof:
                    prof = True
            if prof== True:
                return db.d20.roll() + self.proficiency_bonus + abl_mod
            else:
                return db.d20.roll() + abl_mod
        
    #def skill_check(self, skill):
    
    #def dash(self, movement):
    
    #def disengage(self):
    
    #def dodge(self.advantage_toggle):
    
    #def help(self,target):
    
    #def hide(self, stealth_check):
    
    #def ready(self, trigger, action):
    
    #def search(self, perception_check):
    
    #def use_an_object(self):
    
    #def imporovise(self, skill_check):

In [5]:
class Pc(Creature):
    def __init__(self):
        pass
    def create_pc(self, creature):
        creature.entity_type = 'pc'
        creature.disposition = 'ally'
        setattr(creature, 'pc', True)
        setattr(creature, 'pc_lvl', 1)
        setattr(creature, 'proficiency_bonus', pc_classes.prof_bonus[creature.pc_lvl])
        creature.skills = copy.deepcopy(skills)
        creature.initiative_bonus = creatures.ability_mod_tbl[creature.abilities['dex']]
        if creature.armor == False and creature.ac == False:
            dex_mod = creatures.ability_mod_tbl[creature.abilities['dex']]
            wis_mod = creatures.ability_mod_tbl[creature.abilities['wis']]
            con_mod = creatures.ability_mod_tbl[creature.abilities['con']]
            barb_ua = 10 + con_mod + dex_mod
            monk_ua = 10 + dex_mod + wis_mod
            if 'barbarian' in creature.pc_classes.keys() & 'monk' in creature.pc_classes.keys():
                unarmored_defense = []
                unarmored_defense.append(barb_ua)
                unarmored_defense.append(monk_ua)
                creature.ac = max(unarmored_defense)
            elif 'barbarian' in creature.pc_classes.keys():
                creature.ac = barb_ua
            elif 'monk' in creature.pc_classes.keys():
                creature.ac = monk_ua
            else:
                creature.ac = 10 + creatures.ability_mod_tbl[creature.abilities['dex']]
        

In [6]:
# keeping pc_class and pc as different classes to account for multi-classing

class PcClass:
    def __init__(self):
        pass
    
    def __hill_dwarf_hp_bonus(self, creature):
        creature.hp_max += 1
        
    def add_first_class(self, creature, pc_class, gen_method):
        p_class = pc_class['pc_class']
        setattr(creature, 'pc_classes', {p_class:1})
        setattr(creature, 'abilities', gen_method)
        
        ### adds first class attrs to creature
        for key in pc_class.keys():
            if key=='pc_class':
                pass
            else:
                setattr(creature, key, pc_class[key])
                
        ### adds racial bonuses to ability scores 
        for key in creature.ability_score_increase.keys():
            creature.abilities[key] += creature.ability_score_increase[key]
        
        #Turns creature with class into pc
        Pc.create_pc(self, creature)
        
        #move class profs into creature profs
        for prof in creature.arm_weap_prof:
            creature.proficiencies.append(prof)
            
        
        ### sets hp_max
        con_bonus = creatures.ability_mod_tbl[creature.abilities['con']]
        hp_max = int(pc_class['hit_die'][1:]) + con_bonus
        creature.hp_max = hp_max
        if creature.subrace == 'hill':
            self.__hill_dwarf_hp_bonus(creature)
        creature.hp = hp_max
            
    def add_class_lvl(self, creature, p_class_name):
        ### update hp_max
        class_hp_add = dice_avg[creature.hit_die]
        con_bonus = creatures.ability_mod_tbl[creature.abilities['con']]
        creature.hp_max += class_hp_add + con_bonus
        if creature.subrace == 'hill':
            self.hill_dwarf_hp_bonus(creature)
        
        creature.pc_classes[p_class_name] += 1
        creature.pc_lvl += 1
        creature.proficiency_bonus = pc_classes.prof_bonus[creature.pc_lvl]


In [7]:
class Npc():
    def __init__(self):
        pass
    def is_npc(self, creature):
        creature.entity_type = 'npc'
        creature.entity_type = 'indifferent'
        setattr(creature, 'npc', True)
        setattr(creature, 'max_hp', 0)

In [8]:
class item:
    def __init__(self):
        pass
    

In [9]:
class CombatMechanics:
    def __init__(self, combatants, e_combatants):
        self.round = 1
        for c in e_combatants:
            c.disposition = 'hostile'
        self.combatants = combatants + e_combatants
        #self.c_area = c_area

    def __order_combat(self):
        for i in range(1, len(self.combatants)):
            key_item = self.combatants[i]
            key_item_num = key_item.initiative
            
            j = i-1
            
            while j>=0 and self.combatants[j].initiative < key_item_num:
                self.combatants[j+1] = self.combatants[j]
                j -= 1
                
            self.combatants[j + 1] = key_item
        
    def initiative(self):
        for c in self.combatants:
            advantage = hasattr(c, 'advantage')
            disadvantage = hasattr(c, 'disadvantage')
            init_advantage = False
            init_disadvantage = True
            
            if advantage == True:
                if 'initiative' or 'dex' in c.advantage:
                    init_advantage = True
            if disadvantage == True:
                if 'initiative' or 'dex' in c.disadvantage:
                    init_disadvantage = True
            if init_advantage == True and init_disadvantage == True:
                setattr(c, 'initiative', db.d20.roll() + c.initiative_bonus + c.abilities['dex']/100)
            elif init_advantage == True:
                setattr(c, 'initiative', db.roll_advantage() + c.initiative_bonus + c.abilities['dex']/100)
            elif init_disadvantage == True:
                setattr(c, 'initiative', db.roll_disadvantage() + c.initiative_bonus + c.abilities['dex']/100)
            else:
                setattr(c, 'initiative', db.d20.roll() + c.initiative_bonus + c.abilities['dex']/100)
        self.__order_combat()
    def get_target(self, attacker):
        combatants = self.combatants
        for i, c in enumerate(combatants):
            if c.hp <= 0:
                if i == len(self.combatants):
                    return False
                else:
                    pass
            elif c.disposition == attacker.disposition:
                if i == len(self.combatants):
                    return False
                else:
                    pass
            else:
                return c
            
    def apply_hp(self, target, hp):
        target.hp = target.hp + hp
        
            
    def attack_action(self, attacker, strategy = False):
        target = self.get_target(attacker)
        if target == False or target is None:
            return target
        else:
            a_roll = attacker.attack_roll(attacker.main_hand)
            if a_roll >= target.ac:
                o_hp = target.hp
                dmg = attacker.damage_roll(attacker.main_hand)
                self.apply_hp(target, dmg)
                return {'name': attacker.name, 'target': target.name, 'a_roll':a_roll,
                       'damage': dmg}
            else:
                return {'name': attacker.name, 'target': target.name, 'a_roll':a_roll}
                
    def death_save(self, pc):
        r = db.d20.roll()
        if r == 1:
            pc.death_save_count['f'] += 2
            if pc.death_save_count['f'] >= 3:
                pc.life_state = 'dead'
        elif r == 20:
            pc.death_save_count['s'] += 2
            if pc.death_save_count['s']>= 3:
                pc.life_state = 'stable'
        elif r < 10:
            pc.death_save_count['f'] += 1
            if pc.death_save_count['f'] >= 3:
                pc.life_state = 'dead'
        else:
            pc.death_save_count['s'] += 1
            if pc.death_save_count['s']>= 3:
                pc.life_state = 'stable'            

    def dying(self, pc):
        if pc.disposition == 'ally':
            if pc.life_state == 'alive':
                pc.life_state = 'dying'
                setattr(pc, 'death_save_count', {'s': 0, 'f': 0})
                self.death_save(pc)
            elif pc.life_state == 'dying':
                self.death_save(pc)
                
    def combat(self):
        rnd = 1
        alive = True
        winner = 0
        while alive==True:
            for c in self.combatants:
                hp = c.hp
                c_disposition = c.disposition 
                if hp <= 0:
                    if c_disposition == 'ally':
                        self.dying(c)
                    else:
                        c.life_state = 'dead'
                        pass
                else:
                    a = self.attack_action(c)
                    if a==False:
                        alive = False
                    elif a is None:
                        alive = False
            rnd += 1
        pc_death = [c.life_state for c in self.combatants if c.disposition == 'ally']
        if 'alive' in pc_death:
            winner = 1
        else:
            winner = -1
        dead_count = len([state for state in pc_death if state == 'dead'])
        return winner, dead_count, pc_death
    
    def test_combat(self, num_tests):
        results = []
        death_counts = []
        death_details = []

        for _ in range(num_tests):
            gc = copy.deepcopy([c for c in self.combatants if c.disposition == 'ally'])
            ec = copy.deepcopy([c for c in self.combatants if c.disposition == 'hostile'])
            c_bat = CombatMechanics(gc, ec)
            c_bat.initiative()
            c = c_bat.combat()
            results.append(c[0])
            death_counts.append(c[1])
            death_details.append(c[2])
            
        difficulty = sum(results)/len(results)
        return results, death_counts, death_details, difficulty
    
        

In [10]:
tiny = False
tiny = Creature(races.dwarf_hill, 'tiny')
pc_class = PcClass()

In [11]:
tiny.abilities

False

In [12]:
pc_class.add_first_class(tiny, pc_classes.barbarian, pc_classes.barbarian['ability_by_method']['pb'])

In [13]:
### consume spell slot command

In [14]:
tiny.equip_weapon(wpn['greataxe'])
tiny1 = copy.deepcopy(tiny)
tiny2 = copy.deepcopy(tiny)
tiny3 = copy.deepcopy(tiny)
evil_tiny = copy.deepcopy(tiny)
evil_tiny.disposition = 'hostile'
evil_tiny.name = 'evil_tiny'

In [15]:
aar = Creature(creatures.aarakocra)
aar.equip_weapon(wpn['javelin'], versatile = True)
aar1 = copy.deepcopy(aar)
aar2 = copy.deepcopy(aar)
aar3 = copy.deepcopy(aar)

In [16]:
g = [tiny, tiny1, tiny2, tiny3]
e = [evil_tiny, aar, aar1, aar2, aar3]

In [17]:
cmbt = CombatMechanics(g, e)

In [18]:
tests = cmbt.test_combat(10000)

In [19]:
len(tests)

4

In [20]:
tests[3]

0.7592

In [21]:
d = len([t for t in tests[1] if t>0])
cd = d/len(tests[1])
cd

0.3491

# Playing with creating area
 - currently map is working like a table with the first thre columns being the x, y, z coordinates respectively and the rest of the columns containing information about the specific coordinate.
 - table headings of the matrix would be x, y, z, Terrain features, construction features, other objects, creature 

In [34]:
class AreaMap:
    def __init__(self):
        pass
    def create_map(self, tup: tuple):
        y = tup[0]*tup[1]*tup[2]
        m = np.zeros((6, y), dtype= int)   
        return self.__add_coordinates(m, tup)
    
    def __add_coordinates(self, area, tup):
        x = []
        y = []
        z = []
        coords = []
        for i in range(tup[1]):
            for j in range(tup[0]):
                for k in range(tup[2]):
                    x.append(j)
                    y.append(i)
                    z.append(k)
                    coords.append((j, i, k))
        area[0] = x
        area[1] = y
        area[2] = z
        return area, coords
        
    def __incriment_pos(self, num):
        if num == 0:
            return num, num + 1
        else:
            return num, num +1, num-1
    
    def __axis_adjs(self, tup):
        x_adj = []
        y_adj = []
        z_adj = []
        for i in range(len(tup)):
            ps = self.__incriment_pos(tup[i])
            for j in ps:
                if i == 0:
                    x_adj.append(ps)
                elif i == 1:
                    y_adj.append(ps)
                else:
                    z_adj.append(ps)
        return (x_adj, y_adj, z_adj)
    
    def get_adjs(self, tup: tuple):
        ps = self.__axis_adjs(tup)
        adjs = []
        exists = False
        for i in ps[0]:
            for j in ps[1]:
                for k in ps[2]:
                    for perm in adjs:
                        if perm == (i, j, k):
                            exists = True
                    if exists == True:
                        exists = False
                        next
                    else:
                        adjs.append((i, j, k))
        return adjs
    
    def add_object(self, area, tup: tuple, obj_key):
        area[tup[0]][tup[1]][tup[2]] = obj_key
    
    #def add_creature(self, area, creatures):
    #    self.creatures = []
    #    pg = PINGenerator()
    #    for c in creatures:
    #        pin = pg('c')
    #        if pin in self.creatures:
                
    

In [45]:
area_factory = AreaMap()

In [46]:
a = area_factory.create_map((20, 6, 20))

In [47]:
len(a[1])

2400

In [ ]:
area_factory = AreaMap()
area_1 = area_factory.create_map((20, 6, 20))

In [ ]:
area_factory.add_object(area_1, (0, 0, 0), 1)

In [ ]:
area_1[0][0][0]